# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install pip install datasets scikit-learn peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 32.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from datasets import load_dataset
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [35]:
# I chose spam classifcation -> 2 labels + dicts (for readability as it was in the exmaples)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0:'NOT SPAM', 1:'SPAM'},
    label2id = {"NOT SPAM": 0, 'SPAM':1})

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_set = load_dataset("sms_spam", split='train').train_test_split(
shuffle=True, seed=123, test_size=0.2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for sms_spam contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sms_spam
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [36]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [37]:
data_set['test']

Dataset({
    features: ['sms', 'label'],
    num_rows: 1115
})

In [38]:
splits=['train','test']

In [39]:
## Firstly tokenizing
data_tokenized = {}
for split in splits:
    data_tokenized[split] = data_set[split].map(lambda x: tokenizer( x['sms'],truncation=True,
                                      padding='max_length',  return_tensors="pt"), batched=True)
    data_tokenized[split].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [40]:
batch_size = 2
train_loader = DataLoader(data_tokenized['train'], batch_size=batch_size)
test_loader = DataLoader(data_tokenized['test'], batch_size=batch_size)

In [10]:
def evaluate_results(model_, data_loader):
  predictions = []
  labels = []
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model_ = model_.to(device)
  with torch.no_grad():
      for batch in tqdm(data_loader):
          input_ids = batch['input_ids']
          attention_mask = batch['attention_mask']
          input_ids =input_ids.to(device)
          attention_mask =attention_mask.to(device)
          outputs = model_(input_ids=input_ids, attention_mask=attention_mask)
          prediction = torch.argmax(outputs.logits, dim=-1)

          if device !='cpu':
              prediction =prediction.to('cpu')
          prediction = prediction.numpy()
          label = batch['label'].numpy()

          labels.extend(label)
          predictions.extend(prediction)
      return accuracy_score(labels,predictions)


In [11]:
evaluate_results(model,test_loader)

100%|██████████| 558/558 [00:18<00:00, 30.02it/s]


0.809865470852018

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

Now I will try approach with Trainer


In [41]:
from peft import LoraConfig,TaskType
from peft import get_peft_model
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

In [67]:

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0:'NOT SPAM', 1:'SPAM'},
    label2id = {"NOT SPAM": 0, 'SPAM':1})


config  = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=4,
    target_modules=['distilbert.transformer.layer.5.ffn.lin1']
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
lora_model = get_peft_model(model=model,peft_config=config)

In [69]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels,predictions)}



In [70]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./results",
        learning_rate=0.001,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
        label_names=["labels"]
    ),
    train_dataset=data_tokenized["train"],
    eval_dataset=data_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics

)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.061300,0.032253,0.992825


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=1115, training_loss=0.07669701939741058, metrics={'train_runtime': 109.568, 'train_samples_per_second': 40.696, 'train_steps_per_second': 10.176, 'total_flos': 598993557786624.0, 'train_loss': 0.07669701939741058, 'epoch': 1.0})

In [18]:
trainer.evaluate()['eval_accuracy']

0.9928251121076234

Check using the old function - for curiosity reason

In [73]:
evaluate_results(lora_model,test_loader)

100%|██████████| 558/558 [00:18<00:00, 30.01it/s]


0.9928251121076234

In [71]:
lora_model.save_pretrained("my_lora_test1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [72]:
from peft import AutoPeftModelForSequenceClassification, PeftModel
lora_model1 = AutoPeftModelForSequenceClassification.from_pretrained('my_lora_test1')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
evaluate_results(lora_model1,test_loader)


100%|██████████| 558/558 [00:21<00:00, 25.86it/s]


0.9928251121076234